In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_community.tools.pubmed.tool import PubmedQueryRun

llm = ChatOpenAI(openai_api_key=os.getenv("API_KEY"))

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    Chat History:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

articles = PubmedQueryRun()

tools = [
    Tool.from_function(
        name="PubMed Search",
        description="Use when needing to find a biomedical publications and reseach articles. The question will include the word 'genes', 'disease' and 'proteins. Return a link and abstract to a publication.",
        func=articles.run,
        return_direct=True,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True,
)

while True:
    q = input("> ")
    response = agent_executor.invoke({"input": q})
    print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: PubMed Search
Action Input: role of BTK genePublished: 2024-03-16
Title: Long-read single-cell RNA sequencing enables the study of cancer subclone-specific genotype and phenotype in chronic lymphocytic leukemia.
Copyright Information: 
Summary::
['BTK', 'BTK', 'BTK-', 'BTK', 'BTK', 'BTK']
['C481S', 'C481S', 'C481S']
Bruton's tyrosine kinase (BTK) inhibitors are effective for the treatment of chronic lymphocytic leukemia (CLL) due to BTK's role in B cell survival and proliferation. Treatment resistance is most commonly caused by the emergence of the hallmark  mutation that inhibits drug binding. In this study, we aimed to investigate whether the presence of additional CLL driver mutations in cancer subclones harboring a  mutation accelerates subclone expansion. In addition, we sought to determine whether mutated subclones exhibit distinct transcriptomic behavior when compared to other cancer subclones.